In [1]:
## Comparison of Gerosa, Core and Genome Scale Metabolic Models
% reset
import tellurium as te; te.setDefaultPlottingEngine('matplotlib')
% matplotlib inline
from optlang.symbolics import Zero
import cobra.util.solver as sutil
import os, re, cobra, shutil, cobra, time, json, pickle
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import libsbml
from libsbml import *

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
### create time stamped output folder
stamp = time.strftime("%Y_%m_%d")
output = os.getcwd()+'/output/'+stamp+'/'
input_out = output+'input/'
if not os.path.isdir(os.getcwd()+'/output'):
    os.mkdir(os.getcwd()+'/output')
if not os.path.isdir(output):
    os.mkdir(output)
if not os.path.isdir(input_out):
    os.mkdir(input_out)

## save a copy of this script and all input files in the output folder
def copytree(src, dst, symlinks=False, ignore=None):
    for item in os.listdir(src):
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        if os.path.isdir(s):
            shutil.copytree(s, d, symlinks, ignore)
        else:
            shutil.copy2(s, d)

copytree(os.getcwd()+'/input/',input_out)
shutil.copy2(os.getcwd()+'/parsing_data.ipynb',output)

'/Users/Zarathustra/ETH/Mechanistic_modeling/Pipeline/output/2018_03_13/parsing_data.ipynb'

In [136]:
# encoding = ['utf-8', "cp1252", "ISO-8859-1"]
# df = pd.read_csv(input_out+'km.csv', encoding="ISO-8859-1")
# pd.read_csv(input_out+'regulation.csv', encoding="ISO-8859-1")
# pd.read_csv(input_out+'turnover.csv', encoding='utf-8')

# k = pd.read_csv('input/km.csv', index_col=0, encoding='latin-1')
km = pd.read_csv(input_out+'km.csv', index_col=0, encoding='ISO-8859-1')

value_col = 'KM_Value'
organism = 'Escherichia coli'
km_coli = km[km['Organism'].str.lower() == organism.lower()]
print((km_coli[value_col] > 0).sum())

20613


In [140]:
# convert EC_number to bigg.reaction
# generate table with bigg.reaction_bigg.metabolite as id

km_filtered = km_coli[(pd.isnull(km_coli['Commentary'])) |
      ((km_coli['Commentary'].str.find('mutant') == -1) &
       (km_coli['Commentary'].str.find('mutation') == -1) &
       (km_coli['Commentary'].str.find('variant') == -1) &
       (km_coli['Commentary'].str.find('genetically engineered') == -1))]

print((km_filtered[value_col] > 0).sum())

11855


In [138]:
km = km_filtered[pd.notnull(km_filtered['bigg.metabolite'])].copy()
km['bigg.metabolite'] = km['bigg.metabolite'].str.lower()
km_val = km[km[value_col] > 0]
print(km_val.shape[0])
print(km_val.groupby(('bigg.metabolite', 'EC_number')).first().shape[0])
print(km_val.groupby('bigg.metabolite').first().shape[0])
print(km_val.groupby('EC_number').first().shape[0])

9534
1644
542
649


In [146]:
km_val.groupby(('bigg.metabolite', 'EC_number')).first()


Organism  LigandID   KM_Value  \
bigg.metabolite EC_number                                          
10fthf          2.1.2.2    Escherichia coli     16154    0.07750   
12ppd__s        1.1.1.77   Escherichia coli     20321    1.25000   
14dhncoa        3.1.2.28   Escherichia coli     85376    0.00800   
1pyr5c          1.2.1.41   Escherichia coli     45501    2.50000   
                1.5.1.2    Escherichia coli      1804    0.14000   
23camp          3.1.4.16   Escherichia coli       942    0.49000   
                3.1.4.37   Escherichia coli       942    0.49000   
                3.1.4.55   Escherichia coli       942    0.11000   
23ccmp          3.1.4.55   Escherichia coli      7083    0.14000   
23cgmp          3.1.4.16   Escherichia coli      5089    1.60000   
                3.1.4.37   Escherichia coli      5089    1.60000   
                3.1.4.55   Escherichia coli      5089    0.31000   
23dappa         4.1.99.1   Escherichia coli     46195    0.60000   
                4.3.1.15   Escherichia coli     20756    0.03000   
23ddhb          1.3.1.28   Escherichia coli     29329    0.30000   
23dhb           2.7.7.58   Escherichia coli      3773    0.00270   
                6.3.2.14   Escherichia coli      1367    0.00250   
23dhdp          1.17.1.8   Escherichia coli      4593    0.00900   
24dhhed         4.1.2.20   Escherichia coli     18858    0.10000   
                4.1.2.52   Escherichia coli     10416    0.35000   
                4.1.2.53   Escherichia coli     10416    0.10000   
26dap__m        4.1.1.20   Escherichia coli     20910    1.70000   
                5.1.1.7    Escherichia coli      4267    0.36000   
                6.3.2.13   Escherichia coli      1400    0.03600   
26dap_ll        5.1.1.7    Escherichia coli      8243    0.16000   
2ahbut          1.1.1.86   Escherichia coli      5230    0.00200   
2cpr5p          4.1.1.48   Escherichia coli       967    0.00034   
2dda7p          4.2.3.4    Escherichia coli      4386    0.00055   
2ddg6p          4.1.2.14   Escherichia coli      1596    0.06000   
2ddglcn         2.7.1.45   Escherichia coli      1172    1.00000   
...                                     ...       ...        ...   
uri             2.7.1.48   Escherichia coli       261    0.35000   
                3.2.2.3    Escherichia coli       261    0.40800   
                3.2.2.8    Escherichia coli       261    0.12000   
utp             2.3.1.157  Escherichia coli        65    0.10000   
                2.7.1.105  Escherichia coli        65    0.78000   
                2.7.1.11   Escherichia coli        65    5.10000   
                2.7.1.7    Escherichia coli        65    2.00000   
                2.7.4.22   Escherichia coli        65    0.04600   
                2.7.7.23   Escherichia coli        65    0.10000   
                2.7.7.38   Escherichia coli        65    0.88000   
                2.7.7.9    Escherichia coli        65    0.05800   
                6.3.4.2    Escherichia coli        65    0.02700   
                6.3.4.4    Escherichia coli        65    2.06000   
val__l          2.6.1.42   Escherichia coli       246    2.70000   
                6.1.1.9    Escherichia coli       246    0.00430   
                6.3.2.8    Escherichia coli       835   70.00000   
xan             2.4.2.22   Escherichia coli       234    0.03050   
                2.4.2.29   Escherichia coli       234    0.22400   
xmp             6.3.5.2    Escherichia coli      1007    0.02900   
xtp             6.3.4.4    Escherichia coli      1941    0.38800   
xtsn            2.4.2.1    Escherichia coli      1222    0.05100   
                3.2.2.3    Escherichia coli      1222    0.45400   
xu1p__d         4.1.2.19   Escherichia coli     30788    0.20000   
xu5p__d         2.2.1.1    Escherichia coli       644    0.16000   
xyl__d          1.1.5.2    Escherichia coli       115   17.00000   
                5.1.3.3    Escherichia coli       115   23.00000   
xylt            2.7.1.1